## This script uploads manually downloaded data to AWS bucket for the California Climate Risk Index

In [ ]:
# Import libraries
import boto3
import pandas as pd
import os
import sys

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import append_metadata

In [ ]:
# Set AWS credentials
s3 = boto3.resource('s3')
s3_cl = boto3.client('s3') # for lower-level processes
bucket_name = 'ca-climate-index'
raw_path = '1_pull_data/' # path to raw datafiles in AWS bucket

In [ ]:
def aws_datasource_dirs(domain, datasource):
    """Creates a dir in the respective domain dir, if not already available"""
    bucket = s3.Bucket(bucket_name)

    # path to folder in aws
    datasource_dir = '{0}{1}/{2}/'.format(raw_path, domain, datasource)

    # # check if folder already exists
    dirs = []
    for item in bucket.objects.filter(Prefix=raw_path+domain+'/'):
        d = str(item.key)
        dirs += [d]

    if datasource_dir not in dirs:
        print('Creating folder for {}'.format(datasource_dir))
    #     bucket.put_object(Key=datasource_dir)

    return datasource_dir

@append_metadata
def manual_to_aws(domain, datasource, loc, export=False, varname=''):
    """
    Uploads data that was manually downloaded to AWS bucket.
    
    Parameters
    ----------
    domain: string
        built_environment, governance, natural_systems, society_economy, climate_risk
    datasource: string
        Organization of datasource
    loc: string
        Local path to filename to upload
    export: bool
        If True, exports file to specified AWS bucket
    
    Script
    ------
    manual_pull_upload.ipynb
    '''
    """
    path_to_save = aws_datasource_dirs(domain, datasource)
    print('{0} saved to {1}'.format(fname, path_to_save))
    if export ==True:
        # first check that folder is not already available
        path_to_save = aws_datasource_dirs(domain, datasource)

        # extract the filename from path
        loc = loc.replace('\\', '/')
        fname = loc.split('/')[-1]

        # point to location of file(s) locally and upload to aws
        try:
            s3_cl.upload_file(
                loc,
                bucket_name,
                aws_datasource_dirs(domain, datasource)+fname
            )
            print('{0} saved to {1}'.format(fname, path_to_save))
        except Exception as e:
            print(e)

### Pulling data pipeline file to obtain all variable names for metadata generation

In [ ]:
ref_file = sys.path[-1]+'/metadata/Full Data Pipeline Notes - 1_ Pull.csv'
df = pd.read_csv(ref_file)
# drop empty columns
df = df.loc[:, df.columns.notna()]

df = df.drop(columns=['Link','Unnamed: 15'])
ref_df = df.fillna('N/A')

In [ ]:
pd.set_option('display.max_rows', None)
#ref_df

### Enter in metrics that were scraped and adjusted before uploading to AWS, as they have their own metadata

In [ ]:
# Build list of file names excluding 'N/A'
variable_names = [name for name in df['Variable'].values if name != 'N/A']

# Define problematic files which we are still investigating
skip_vars = ['natural_epa_air_quality',
             'governance_edd_responder_firefighter',
             'governance_edd_responder_nurse',
             'governance_edd_responder_parametics',
             'governance_edd_responder_police',
             'climate_noaa_flood_fatalities',
             'climate_usda_heat_crop_loss',
             'climate_usda_heat_crop_cost']

# Exclude files from the list
included_vars = [name for name in variable_names if name not in skip_vars]

### Generating metadata for all metrics that were manually downloaded and uploaded to AWS
* loop through each variable name not excluded and generate metadata

In [ ]:
for var in included_vars:
    # get the file name by itself (no subdirectories)
    varname = "test"
    print(varname)
    # this is just for metadata creation, so export is set to false and the first
    # three variables can be anything
    manual_to_aws('all', 'all', 'any', export=False, varname=varname)
    raise Exception